In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os


In [ ]:
data_path = os.path.join("..", "data")

In [ ]:
X = np.load(data_path + '/initial_data/function_8/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_8/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
# add the week 1 input and  result to the array
X = np.append(X, [[0.022440 , 0.191554 , 0.027755 ,0.095156 , 0.791674 ,0.395249 ,0.075154,0.444097]], axis=0)
y = np.append(y, [9.907261118], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.008892, 0.448205, 0.002458, 0.605908, 0.634393, 0.706540, 0.036133, 0.701302]], axis=0)
y = np.append(y, [9.5267168451681], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.135865, 0.220521, 0.003626, 0.027379, 0.410930, 0.081034, 0.127128, 0.747297]], axis=0)
y = np.append(y, [9.6654965798451], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.122471, 0.258725, 0.109219, 0.228651, 0.547005, 0.439823, 0.346755, 0.502501]], axis=0)
y = np.append(y, [9.9000382811174], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.192695, 0.299481, 0.178516, 0.223527, 0.635056, 0.453264, 0.202099, 0.598408]], axis=0)
y = np.append(y, [9.9322064977796], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.128221, 0.210897, 0.140317, 0.254160, 0.652099, 0.458573, 0.200614, 0.567749]], axis=0)
y = np.append(y, [9.9707717627204], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.116710, 0.257625, 0.102652, 0.256898, 0.608380, 0.438972, 0.202376, 0.673922]], axis=0)
y = np.append(y, [9.9515462235146], axis=0)

# add the week 8 input and result to the array
X = np.append(X, [[0.100296, 0.265682, 0.146924, 0.288241, 0.662098, 0.485298, 0.240507, 0.639099]], axis=0)
y = np.append(y, [9.9534885233509], axis=0)

# add the week 9 input and result to the array
X = np.append(X, [[0.960000, 0.970000, 0.660000, 0.660000, 0.670000, 0.910000, 0.460000, 0.560000]], axis=0)
y = np.append(y, [6.43369], axis=0)

# add the week 10 input and result to the array
X = np.append(X, [[0.096247, 0.307438, 0.179137, 0.224319, 0.639023, 0.468022, 0.244508, 0.623882]], axis=0)
y = np.append(y, [9.9444201092011], axis=0)

# add the week 11 input and result to the array
X = np.append(X, [[0.097805, 0.249076, 0.092150, 0.283382, 0.631658, 0.456355, 0.291001, 0.613192]], axis=0)
y = np.append(y, [9.9354315173546], axis=0)

# add the week 12 input and result to the array
X = np.append(X, [[0.185782, 0.257161, 0.122317, 0.261284, 0.594025, 0.405767, 0.247267, 0.624735]], axis=0)
y = np.append(y, [9.926615973706], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
print (f"max y is :{np.max(y)}")

In [ ]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

In [ ]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
def ucb(x, gp, kappa=4.0):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
# not used in the code
"""grid_size = 10
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
x3 = np.linspace(0, 1, grid_size)
x4 = np.linspace(0, 1, grid_size)
x5 = np.linspace(0, 1, grid_size)
x6 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b, c, d, e, f] for a in x1 for b in x2 for c in x3 for d in x4 for e in x5 for f in x6])"""

In [ ]:
n_samples = 3000000  # You can adjust this based on your compute budget

# Generate random 6D points in [0, 1]^8
X_random = np.random.rand(n_samples, 8)

In [ ]:
ucb_values = np.array([ucb(x, gp) for x in X_random])


In [ ]:
best_idx = np.argmax(ucb_values)
# next_best_point = X_grid[best_idx]
next_best_point = X_random[best_idx]
print("Next best point to evaluate (UCB) rounded:", np.round(next_best_point,6))
print("Next best point to evaluate (UCB):", next_best_point)
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Predicted maximum value (mean):", pred_mean[0])

In [ ]:
# SVM model as surrogate
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 8)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)